In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from hazm import *
from nltk.tokenize import word_tokenize
import re

In [2]:
# read data
df = pd.read_csv("Task/Topic Modeling/twitter_corona981_full/exportData/corona981/twitter_corona981_full.csv")
df.head()

,Unnamed: 0,favorite_count,hashtagField,location,mentionField,ownerId,ownerName,postId,reply_count,retweet_count,retweeted_ownerId,retweeted_postId,tagField,textField,textField_normal,time_jalali,timestamp
0,0,"""0""","""ویروس_کرونا""","""جمهوری اسلامی ایران""","""""",1140945568505708544,"""Safire_Mehr""",1221133198379192321,"""0""","""0""","""0""","""0""","""""","""کرونا داشتیم وقتی کرونا مد نبود #ویروس_کرونا""","""کرونا داشتیم وقتی کرونا مد نبود ویروس کرونا""",1398-11-05 21:40:16,1579975816
1,1,"""0""","""ویروس_کرونا;کرونا""","""""","""""",1221925473900429313,"""Jonny_walker666""",1223016973975674881,"""0""","""0""","""0""","""0""","""""","""گفتم کرونا ناراحت نشه یه موقع #کرونا #ویروس_ک...","""گفتم کرونا ناراحت نشه یه موقع کرونا ویروس کرونا""",1398-11-11 02:25:43,1580424943
2,2,"""3""","""ویروس_کرونا""","""""","""""",1214838745918259200,"""rzRz471""",1220858435044093952,"""0""","""1""","""0""","""0""","""ویروس کرونا""","""ویروس کرونا داشتیم وقتی کرونا مد نبود #ویروس_...","""ویروس کرونا داشتیم وقتی کرونا مد نبود ویروس ک...",1398-11-05 03:28:28,1579910308
3,3,"""0""","""""","""""","""""",866686549722845184,"""laleh_wazhegoon""",1220980152122953729,"""0""","""0""","""0""","""0""","""""","""اینجا کرونا آنجا کرونا همه جا کرونا این کرونا...","""اینجا کرونا انجا کرونا همه جا کرونا این کرونا...",1398-11-05 11:32:07,1579939327
4,4,"""0""","""""","""""","""seyedmohammadww""",1182218416012312576,"""VTPmKSO7wFxM8gZ""",1220760647232901120,"""0""","""0""","""1110607722624573441""","""1220669177826041856""","""ویروس کرونا""","""RT @seyedmohammadww: اسلام از ویروس کرونا کشن...","""rt seyedmohammadww اسلام از ویروس کرونا کشنده...",1398-11-04 20:59:53,1579886993


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67022 entries, 0 to 67021
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         67022 non-null  int64 
 1   favorite_count     67022 non-null  object
 2   hashtagField       67022 non-null  object
 3   location           67022 non-null  object
 4   mentionField       67022 non-null  object
 5   ownerId            67022 non-null  int64 
 6   ownerName          67022 non-null  object
 7   postId             67022 non-null  int64 
 8   reply_count        67022 non-null  object
 9   retweet_count      67022 non-null  object
 10  retweeted_ownerId  67022 non-null  object
 11  retweeted_postId   67022 non-null  object
 12  tagField           67022 non-null  object
 13  textField          67022 non-null  object
 14  textField_normal   67022 non-null  object
 15  time_jalali        67022 non-null  object
 16  timestamp          67022 non-null  int64

In [60]:
tweets = df['textField_normal']
tweets = tweets.drop_duplicates().reset_index(drop=True)

In [61]:
# remove english words from persian tweets
def remove_english(tw):
    words = word_tokenize(tw)
    words = " ".join(w for w in words if not re.match(r'[a-zA-Z0-9]+', w))
    return words
tweets = tweets.apply(lambda x: remove_english(x))  

In [62]:
stopwords = utils.stopwords_list()
# remove stopwords
def remove_stop(tw):
    words = word_tokenize(tw)
    words = " ".join(w for w in words if w not in stopwords)
    return words
tweets = tweets.apply(lambda x: remove_stop(x))

In [64]:
# lemmatize words
lemmatizer = Lemmatizer()
def lemmatize_words(tw):
    words = word_tokenize(tw)
    words = " ".join(lemmatizer.lemmatize(w) for w in words)
    return words
tweets = tweets.apply(lambda x: lemmatize_words(x))